In [1]:
# !pip install transformers
!pip install bitsandbytes -q

import os
import random

import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.cuda.amp import GradScaler

from datasets import load_dataset
from transformers import M2M100ForConditionalGeneration, AutoTokenizer, SiglipImageProcessor
from transformers.modeling_outputs import BaseModelOutput
from huggingface_hub import snapshot_download, login

from PIL import Image
from tqdm import tqdm


# model_name = "mtmlt/sonar-nllb-200-1.3B"
# model_dir = snapshot_download(model_name)

# model = M2M100ForConditionalGeneration.from_pretrained(model_dir)
# tokenizer = AutoTokenizer.from_pretrained(model_dir)

# def encode_mean_pool(texts, tokenizer, encoder, lang='eng_Latn', norm=False):
#     tokenizer.src_lang = lang
#     with torch.inference_mode():
#         batch = tokenizer(texts, return_tensors='pt', padding=True)
#         seq_embs = encoder(**batch)
#         mask = batch.attention_mask
#         mean_emb = (seq_embs.last_hidden_state * mask.unsqueeze(-1)).sum(1) / mask.unsqueeze(-1).sum(1)
#         if norm:
#             mean_emb = torch.nn.functional.normalize(mean_emb)
#     return mean_emb, seq_embs, mask

# sentences = ['My name is SONAR.', 'I can embed the sentences into vector space.']

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 117.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour 

2025-12-22 07:14:04.648136: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766387644.788199      20 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766387644.831104      20 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
model_name = "mtmlt/sonar-nllb-200-1.3B"
model_dir = snapshot_download(model_name)
img_model_dir = snapshot_download("google/siglip-base-patch16-384")

Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/804 [00:00<?, ?B/s]

.gitattributes: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

...

README.md: 0.00B [00:00, ?B/s]

tokenizer_config.json:   0%|          | 0.00/711 [00:00<?, ?B/s]

.gitattributes: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/798k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/814M [00:00<?, ?B/s]

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

decoder_model = M2M100ForConditionalGeneration.from_pretrained(model_dir)
tokenizer = AutoTokenizer.from_pretrained(model_dir)
tokenizer.tgt_lang = 'eng_Latn'
tokenizer.src_lang = 'eng_Latn'
decoder_model.eval()  

decoder_encoder = decoder_model.model.encoder  
decoder_model.eval()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

M2M100ForConditionalGeneration(
  (model): M2M100Model(
    (shared): M2M100ScaledWordEmbedding(256206, 1024, padding_idx=1)
    (encoder): M2M100Encoder(
      (embed_tokens): M2M100ScaledWordEmbedding(256206, 1024, padding_idx=1)
      (embed_positions): M2M100SinusoidalPositionalEmbedding()
      (layers): ModuleList(
        (0-23): 24 x M2M100EncoderLayer(
          (self_attn): M2M100Attention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=8192, bias=True)
          (fc2): Linear(in_features=8192, out_features=1024, bias=True)
       

In [ ]:
import torch.nn as nn

import torch
import torch.nn as nn
from transformers import SiglipVisionModel, get_cosine_schedule_with_warmup

class SonarImageEnc(nn.Module):
    """
    Image encoder that projects SigLIP embeddings into SONAR's text embedding space.
    Key: SONAR text embeddings have std ~0.006-0.007, so we need to match that scale.
    """
    def __init__(self, path="google/siglip-base-patch16-384", sonar_dim=1024, target_std=0.007):
        super().__init__()
        self.vision_encoder = SiglipVisionModel.from_pretrained(path, torch_dtype="auto")
        for p in self.vision_encoder.parameters():
            p.requires_grad = False
        vdim = self.vision_encoder.config.hidden_size
        self.target_std = target_std

        # Project AFTER pooling (like the working old notebook)
        # NO LayerNorm at the end - we need to match SONAR's unnormalized scale
        self.proj = nn.Sequential(
            nn.Linear(vdim, 2048),
            nn.GELU(),
            nn.Dropout(0.1),
            nn.Linear(2048, 2048),
            nn.GELU(),
            nn.Dropout(0.1),
            nn.Linear(2048, sonar_dim)
        )
        
        # Initialize final layer to produce small-scale outputs matching SONAR
        self._init_final_layer()

    def _init_final_layer(self):
        """Initialize final layer to output embeddings with std ~0.007"""
        final_layer = self.proj[-1]
        # Scale down the weights significantly
        nn.init.normal_(final_layer.weight, mean=0.0, std=0.01)
        nn.init.zeros_(final_layer.bias)

    def forward(self, pixel_values):
        # Pool FIRST using pooler_output (like old notebook)
        h = self.vision_encoder(pixel_values).pooler_output  # [B, vdim]
        h = self.proj(h)  # [B, sonar_dim]
        return h  # NO LayerNorm - raw output to match SONAR scale

image_encoder = SonarImageEnc(path=img_model_dir).to(device)

In [5]:
def text_to_sonar_embedding(texts=None, tokenizer=None, encoder_model=None, batch=None, lang='eng_Latn', device='cpu', norm=False):
    if batch is None:
        tokenizer.src_lang = lang
        batch = tokenizer(texts, return_tensors='pt', padding=True, truncation=True)
    batch = {k: v.to(device) for k, v in batch.items()}
    encoder_model.to(device)
    with torch.no_grad():
        seq_embs = encoder_model(**batch)
    mask = batch['attention_mask']
    mean_emb = (seq_embs.last_hidden_state * mask.unsqueeze(-1)).sum(1) / mask.sum(1, keepdim=True)
    if norm:
        mean_emb = F.normalize(mean_emb, dim=-1)
    return mean_emb, seq_embs, mask, batch

for param in decoder_model.parameters():
    param.requires_grad = False

In [6]:
from huggingface_hub import snapshot_download

dset = snapshot_download("romrawinjp/multilingual-coco", allow_patterns=["data/train-00015-of-00028.parquet"], repo_type="dataset")
dset = snapshot_download("romrawinjp/multilingual-coco", allow_patterns=["data/train-00016-of-00028.parquet"], repo_type="dataset")
dset = snapshot_download("romrawinjp/multilingual-coco", allow_patterns=["data/train-00017-of-00028.parquet"], repo_type="dataset")
dset = snapshot_download("romrawinjp/multilingual-coco", allow_patterns=["data/train-00018-of-00028.parquet"], repo_type="dataset")
dset = snapshot_download("romrawinjp/multilingual-coco", allow_patterns=["data/train-00019-of-00028.parquet"], repo_type="dataset")
dset = snapshot_download("romrawinjp/multilingual-coco", allow_patterns=["data/train-00020-of-00028.parquet"], repo_type="dataset")
dset = snapshot_download("romrawinjp/multilingual-coco", allow_patterns=["data/train-00021-of-00028.parquet"], repo_type="dataset")
dset = snapshot_download("romrawinjp/multilingual-coco", allow_patterns=["data/train-00022-of-00028.parquet"], repo_type="dataset")
dset = snapshot_download("romrawinjp/multilingual-coco", allow_patterns=["data/train-00023-of-00028.parquet"], repo_type="dataset")
dset = snapshot_download("romrawinjp/multilingual-coco", allow_patterns=["data/train-00024-of-00028.parquet"], repo_type="dataset")
dset = snapshot_download("romrawinjp/multilingual-coco", allow_patterns=["data/train-00025-of-00028.parquet"], repo_type="dataset")
dset = snapshot_download("romrawinjp/multilingual-coco", allow_patterns=["data/train-00026-of-00028.parquet"], repo_type="dataset")
dset = snapshot_download("romrawinjp/multilingual-coco", allow_patterns=["data/train-00027-of-00028.parquet"], repo_type="dataset")

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

data/train-00015-of-00028.parquet:   0%|          | 0.00/480M [00:00<?, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

data/train-00016-of-00028.parquet:   0%|          | 0.00/496M [00:00<?, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

...

data/train-00025-of-00028.parquet:   0%|          | 0.00/479M [00:00<?, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

data/train-00026-of-00028.parquet:   0%|          | 0.00/497M [00:00<?, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

data/train-00027-of-00028.parquet:   0%|          | 0.00/492M [00:00<?, ?B/s]

In [7]:
dataset_train = load_dataset(dset, streaming=True)

In [8]:
image_processor = SiglipImageProcessor.from_pretrained(img_model_dir)  

def normalize_caption(cap_item):
    """
    Extract first valid string from nested tuple/list structure.
    Returns the string or None if invalid.
    """
    if isinstance(cap_item, (tuple, list)):
        for elem in cap_item:
            if isinstance(elem, str) and elem.strip():
                return elem
        return None
    elif isinstance(cap_item, str) and cap_item.strip():
        return cap_item
    return None

def process_img(batch):    
    lang_to_sonar = {
        "en": "eng_Latn",
        "ar": "arb_Arab",
        "de": "dan_Latn",
        "ru": "rus_Cyrl"
    }
    
    rgb_images = [img.convert("RGB") for img in batch["image"]]
    processed_img = image_processor(images=rgb_images, return_tensors="pt")
    
    return {
        "pixel_values": processed_img["pixel_values"],
        "eng_Latn": [normalize_caption(cap) for cap in batch["en"]],
        "arb_Arab": [normalize_caption(cap) for cap in batch["ar"]],
        "dan_Latn": [normalize_caption(cap) for cap in batch["de"]],
        "rus_Cyrl": [normalize_caption(cap) for cap in batch["ru"]]
    }
    
def process_img_val(batch):    
    rgb_images = [img.convert("RGB") for img in batch["jpg"]]
    processed_img = image_processor(images=rgb_images, return_tensors="pt")
    
    return {
        "pixel_values": processed_img["pixel_values"],
        "caption": batch["txt"]
    }

import os as _os
_hf_token = _os.getenv("HF_TOKEN")
if _hf_token:
    login(_hf_token)
else:
    print("HF_TOKEN not set; proceeding without login.")

dataset_train = load_dataset(dset, streaming=True)
dataset_train = dataset_train.map(process_img, batched=True, batch_size=32, remove_columns=list(next(iter(dataset_train.values())).features)).with_format("torch")

HF_TOKEN not set; proceeding without login.


In [9]:
val_hf_dataset = load_dataset("pixparse/cc12m-wds", data_files="cc12m-train-0999.tar")
val_hf_dataset = val_hf_dataset["train"]
val_hf_dataset = val_hf_dataset.select(range(0, 1000))
val_hf_dataset = val_hf_dataset.map(process_img_val, batched=True, batch_size=128, remove_columns=val_hf_dataset.column_names, keep_in_memory=True)

README.md: 0.00B [00:00, ?B/s]

cc12m-train-0999.tar:   0%|          | 0.00/544M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [10]:
train_hf_dataset = dataset_train["train"]
batch_size = 3
train_loader = DataLoader(train_hf_dataset, batch_size=batch_size)

val_hf_dataset.set_format("torch")
val_loader = DataLoader(val_hf_dataset, batch_size=batch_size, shuffle=False)

In [11]:
num_epochs = 5
save_dir = "./adapter_checkpoints"
os.makedirs(save_dir, exist_ok=True)

In [12]:
import gc

gc.collect()

399

In [ ]:
import torch
import torch.nn.functional as F
from torch import nn
from tqdm import tqdm
import os
import random
import bitsandbytes as bnb
from torch.amp import GradScaler
from transformers import get_cosine_schedule_with_warmup

def validate(
    image_encoder,
    decoder_model,
    val_loader,
    tokenizer,
    device,
    max_batches=None,
    max_new_tokens=48,
    num_beams=5
):
    image_encoder.eval()
    decoder_model.eval()
    
    total_ce_loss = 0.0
    num_batches = 0

    with torch.no_grad():
        for i, batch in enumerate(val_loader):
            if max_batches and i >= max_batches:
                break

            pixel_values = batch['pixel_values'].to(device)
            captions = batch['caption']  

            tokenizer.src_lang = 'eng_Latn'
            tokenized_en = tokenizer(
                captions, return_tensors="pt", padding=True, truncation=True, max_length=128
            ).to(device)
            
            labels = tokenized_en['input_ids'].clone()
            labels[labels == tokenizer.pad_token_id] = -100

            # NO normalization - preserve SONAR-compatible scale
            img_vec = image_encoder(pixel_values)         
            enc_out = BaseModelOutput(last_hidden_state=img_vec.unsqueeze(1)) 

            enc_mask = torch.ones(img_vec.size(0), 1, dtype=torch.long, device=device)

            outputs_en = decoder_model(
                encoder_outputs=enc_out,
                attention_mask=enc_mask,
                labels=labels,
                return_dict=True
            )
            ce_loss = outputs_en.loss
            total_ce_loss += ce_loss.item()
            
            generated_en = decoder_model.generate(
                encoder_outputs=BaseModelOutput(last_hidden_state=img_vec.unsqueeze(1)),
                forced_bos_token_id=tokenizer.convert_tokens_to_ids('eng_Latn'),
                max_new_tokens=max_new_tokens,
                num_beams=num_beams,
            )
            preds_en = tokenizer.batch_decode(generated_en, skip_special_tokens=True)

            generated_bn = decoder_model.generate(
                encoder_outputs=BaseModelOutput(last_hidden_state=img_vec.unsqueeze(1)),
                forced_bos_token_id=tokenizer.convert_tokens_to_ids('ben_Beng'),
                max_new_tokens=max_new_tokens,
                num_beams=num_beams,
            )
            preds_bn = tokenizer.batch_decode(generated_bn, skip_special_tokens=True)

            if i % 50 == 0:
                # Compute per-sample std (correct way)
                img_std = img_vec.std(dim=-1).mean()
                print(f"\n{'='*70}")
                print(f"Batch {i} | Zero-Shot Bengali Inspection")
                print(f"{'='*70}")
                print(f"True EN:       {captions[:2]}")
                print(f"Generated EN:  {preds_en[:2]}")
                print(f"Generated BEN: {preds_bn[:2]}")
                print(f"CE Loss: {ce_loss.item():.3f} | IMG vec std (per-sample): {img_std.item():.5f}")

            num_batches += 1

    avg_ce = total_ce_loss / num_batches if num_batches > 0 else 0

    print(f"\n{'='*70}")
    print(f"VALIDATION SUMMARY")
    print(f"{'='*70}")
    print(f"Total batches: {num_batches}")
    print(f"Avg CE Loss: {avg_ce:.4f} (lower is better)")
    print(f"{'='*70}\n")

    image_encoder.train()
    decoder_model.train()
    
    return avg_ce

In [ ]:
def generate_image_embed(
        image_encoder, 
        decoder_model, 
        pixel_values, 
        labels, 
        device="cuda"
    ):
    """Generate image embeddings - NO normalization to preserve SONAR-compatible scale"""
    img_vec = image_encoder(pixel_values) 

    # NO normalization here! SONAR expects unnormalized embeddings with std ~0.007
    enc_out = BaseModelOutput(last_hidden_state=img_vec.unsqueeze(1))  

    enc_mask = torch.ones(img_vec.size(0), 1, dtype=torch.long, device=device)

    outputs = decoder_model(
        encoder_outputs=enc_out,
        attention_mask=enc_mask,         
        labels=labels,
        return_dict=True
    )

    return outputs, img_vec

def generate_text_embed(
    tokenized,
    encoder_model=None,
    decoder_model=None,
):
    """Generate SONAR text embeddings via mean pooling"""
    with torch.no_grad():  # Text encoder is frozen
        seq_embs = encoder_model(**tokenized)
    mask = tokenized['attention_mask']
    text_vec = (seq_embs.last_hidden_state * mask.unsqueeze(-1)).sum(1) / mask.sum(1, keepdim=True)
    
    text_out = BaseModelOutput(last_hidden_state=text_vec.unsqueeze(1))
    text_mask = torch.ones(text_vec.size(0), 1, dtype=torch.long, device=text_vec.device)
    outputs = decoder_model(
        encoder_outputs=text_out,
        attention_mask=text_mask,         
        labels=tokenized['input_ids'],
        return_dict=True
    )

    return outputs, text_vec

def train_multilingual(
    image_encoder,
    decoder_model,
    tokenizer,
    train_loader,
    val_loader,
    device='cuda',
    max_steps=12000,
    grad_accum=48,
    lr=2e-4,
    eval_every=1000,
    save_dir='./checkpoints',
    log_every=500,
    max_new_tokens=48,
    num_beams=5,
    target_std=0.007  # Target std to match SONAR
):
    image_encoder.train().to(device)
    decoder_model.eval().to(device)
    text_enc = decoder_model.model.encoder
    for p in decoder_model.parameters():
        p.requires_grad = False
    for p in text_enc.parameters():
        p.requires_grad = False

    langs = ["eng_Latn"]

    optimizer = bnb.optim.AdamW8bit(
        filter(lambda p: p.requires_grad, image_encoder.parameters()),
        lr=lr, weight_decay=0.01
    )

    scheduler = get_cosine_schedule_with_warmup(
        optimizer, num_warmup_steps=500, num_training_steps=max_steps
    )

    global_step, best_val = 0, float('inf')
    os.makedirs(save_dir, exist_ok=True)

    while global_step < max_steps:
        epoch = (global_step // 82800) + 1
        pbar = tqdm(train_loader, desc=f"Ep {epoch}")

        for micro_step, batch in enumerate(pbar):
            pixel_values = batch['pixel_values'].to(device, non_blocking=True)
            captions, lang = None, None
            for l in langs:
                if batch.get(l):
                    captions, lang = batch[l], l
                    break
            if captions is None:
                continue

            tokenizer.src_lang = lang
            tok = tokenizer(captions, return_tensors='pt', padding=True,
                            truncation=True, max_length=128).to(device)
            labels = tok.input_ids.clone()
            labels[labels == tokenizer.pad_token_id] = -100

            outputs_text, text_vec = generate_text_embed(
                tokenized=tok,
                encoder_model=text_enc,
                decoder_model=decoder_model,
            )
            
            outputs, img_vec = generate_image_embed(
                image_encoder,
                decoder_model,
                pixel_values,
                labels,
                device=device
            )

            # Alignment loss: MSE to match SONAR embeddings directly (no normalization!)
            loss_mse = F.mse_loss(img_vec, text_vec)
            
            # Cosine similarity loss (complementary to MSE)
            loss_cos = 1 - F.cosine_similarity(img_vec, text_vec).mean()
            
            # Scale regularization: encourage img embeddings to have similar std as text embeddings
            img_std = img_vec.std(dim=-1).mean()  # Per-sample std
            text_std = text_vec.std(dim=-1).mean()
            loss_scale = (img_std - text_std).abs()

            # Combined loss: CE for generation + MSE for alignment + scale matching
            loss = (outputs.loss + loss_mse + 0.1 * loss_cos + 0.5 * loss_scale + 0.1 * outputs_text.loss) / grad_accum

            loss.backward()
            if (micro_step + 1) % grad_accum == 0:
                torch.nn.utils.clip_grad_norm_(image_encoder.parameters(), 1.0)
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad(set_to_none=True)
                global_step += 1

                pbar.set_postfix({
                    'loss': loss.item() * grad_accum, 
                    'mse': loss_mse.item(),
                    'img_std': f"{img_std.item():.4f}",
                    'txt_std': f"{text_std.item():.4f}",
                    'lr': f"{scheduler.get_last_lr()[0]:.2e}"
                })

                if global_step % log_every == 0:
                    print(f"\nloss_mse: {loss_mse.item():.3f}, loss_cos: {loss_cos.item():.3f}, loss_scale: {loss_scale.item():.4f}")
                    print(f"IMG std (per-sample): {img_std.item():.5f} | TXT std (per-sample): {text_std.item():.5f}")
                    with torch.no_grad():
                        enc_out = BaseModelOutput(last_hidden_state=img_vec.unsqueeze(1))
                        enc_mask = torch.ones(img_vec.size(0), 1, dtype=torch.long, device=device)
                        
                        generated_en = decoder_model.generate(
                            encoder_outputs=enc_out,
                            attention_mask=enc_mask,
                            forced_bos_token_id=tokenizer.convert_tokens_to_ids('eng_Latn'),
                            max_new_tokens=max_new_tokens,
                            num_beams=num_beams,
                        )
                        preds_en = tokenizer.batch_decode(generated_en, skip_special_tokens=True)
                        
                        generated_bn = decoder_model.generate(
                            encoder_outputs=BaseModelOutput(last_hidden_state=img_vec.unsqueeze(1)),
                            attention_mask=enc_mask,
                            forced_bos_token_id=tokenizer.convert_tokens_to_ids('ben_Beng'),
                            max_new_tokens=max_new_tokens,
                            num_beams=num_beams,
                        )
                        preds_bn = tokenizer.batch_decode(generated_bn, skip_special_tokens=True)
                    
                    print(f"\n{'='*70}")
                    print(f"Step {global_step} | Training Caption Inspection")
                    print(f"{'='*70}")
                    print(f"True EN:       {captions[:2]}")
                    print(f"Generated EN:  {preds_en[:2]}")
                    print(f"Generated BEN: {preds_bn[:2]}")
                    print(f"{'='*70}\n")
                    
                if global_step % eval_every == 0:
                    val_loss = validate(image_encoder, decoder_model, val_loader,
                                        tokenizer, device, max_batches=30)
                    print(f"\nStep {global_step}  val-loss {val_loss:.4f}")
                    if val_loss < best_val:
                        best_val = val_loss
                        torch.save({'step': global_step,
                                    'encoder': image_encoder.state_dict(),
                                    'opt': optimizer.state_dict()},
                                   f"{save_dir}/best.pt")
                if global_step >= max_steps:
                    break
        if global_step >= max_steps:
            break

    print(f"\nDone  best val-loss {best_val:.4f}")
    return global_step, best_val

In [15]:
global_step, best_loss = train_multilingual(
    image_encoder=image_encoder,
    decoder_model=decoder_model,
    tokenizer=tokenizer,
    train_loader=train_loader,
    val_loader=val_loader,
    device=device,
    max_steps=16000,
    grad_accum=10,
    lr=5e-5,
    eval_every=1000,
    save_dir=save_dir
)

Ep 1: 0it [00:00, ?it/s]/usr/local/lib/python3.11/dist-packages/datasets/formatting/torch_formatter.py:90: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(value, **{**default_dtype, **self.torch_tensor_kwargs})
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.58.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
Ep 1: 4999it [19:20,  4.74it/s, loss=3.65, loss_mse=0.642, loss_text=1.78, loss_img=2.83, lr=5.00e-05]

loss_mse: 0.642, loss_text: 1.779, loss_img: 2.828


Ep 1: 5001it [19:22,  1.60it/s, loss=3.65, loss_mse=0.642, loss_text=1.78, loss_img=2.83, lr=5.00e-05]


Step 500 | Training Caption Inspection
True EN:       ['A single yellow rose sits in a red vase on a shelf.', 'A bunch of white hyacinths arranged in a vase.']
Generated EN:  ['A picture of a flower sitting on a table.', 'A picture of a cake sitting on a table.']
Generated BEN: ['একটি রঙিন ট্যাটু সঙ্গে একটি জানালা উপর একটি ব্লু স্ট্রিপ.', 'একটি রঙিন টালি, যা একটি টালি, একটি টালি, একটি টালি, একটি টালি, একটি টালি, একটি টালি, একটি টালি, একটি টালি, একটি টালি, একটি টালি']
Loss: 3.648 | IMG vec std: 0.03 | TXT vec std: 0.01



Ep 1: 9999it [38:38,  4.44it/s, loss=2.26, loss_mse=0.506, loss_text=0.736, loss_img=1.68, lr=4.99e-05]

...


Step 15500 | Training Caption Inspection
True EN:       ['The owner of this bathroom keeps a Wired Magazine handy.', 'A room with a toilet and medical equipment.']
Generated EN:  ['A bathroom with a toilet seat next to a newspaper.', 'A bathroom with a toilet seat next to a toilet.']
Generated BEN: ['A toilet seat with a book next to a toilet.', 'A toilet seat next to a toilet seat.']
Loss: 3.646 | IMG vec std: 0.03 | TXT vec std: 0.01



Ep 1: 6279it [25:30,  4.48it/s, loss=2.85, loss_mse=0.629, loss_text=1.94, loss_img=2.03, lr=0.00e+00]

loss_mse: 0.629, loss_text: 1.939, loss_img: 2.028

Step 16000 | Training Caption Inspection
True EN:       ['A man riding a wave on top of a surboard.', 'Surfer on wave in ocean with large rocks in foreground']
Generated EN:  ['A man on a surfboard riding on a large wave.', 'Surfer riding on a wave near the ocean.']
Generated BEN: ['A man riding a surfboard on top of a wave', 'Surfer surfer on a big rock near the ocean.']
Loss: 2.851 | IMG vec std: 0.03 | TXT vec std: 0.01


Batch 0 | Zero-Shot Bengali Inspection
True EN:       ['The Living Room at the W Midtown in Atlanta', 'Destination Wedding in Tulum | The Celebration Society']
Generated EN:  ['A room that is filled with many red chairs.', 'A man and woman are jumping into the air together.']
Generated BEN: ['এই রুমটি দেখানো হয় একটি চার্চ, যেখানে অনেক জিনিস আছে।', 'A couple of men and women are jumping into the air together.']
CE Loss: 5.684 | IMG vec std: 0.0313


Ep 1: 6279it [26:19,  3.97it/s, loss=2.85, loss_mse=0.629, loss_text=1.94, loss_img=2.03, lr=0.00e+00]


VALIDATION SUMMARY
Total batches: 30
Avg CE Loss: 5.0139 (lower is better)


Step 16000  val-loss 5.0139

Done  best val-loss 4.7372
